In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import ast

C:\Users\charl\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Flatten, RepeatVector, TimeDistributed
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
note_mapping = {
    'C0': 1, 'C#0': 2, 'D0': 3, 'D#0': 4, 'E0': 5, 'F0': 6, 'F#0': 7, 'G0': 8, 'G#0': 9, 'A1': 10,
    'A#1': 11, 'B1': 12, 'C1': 13, 'C#1': 14, 'D1': 15, 'D#1': 16, 'E1': 17, 'F1': 18, 'F#1': 19,
    'G1': 20, 'G#1': 21, 'A2': 22, 'A#2': 23, 'B2': 24, 'C2': 25, 'C#2': 26, 'D2': 27, 'D#2': 28,
    'E2': 29, 'F2': 30, 'F#2': 31, 'G2': 32, 'G#2': 33, 'A3': 34, 'A#3': 35, 'B3': 36, 'C3': 37,
    'C#3': 38, 'D3': 39, 'D#3': 40, 'E3': 41, 'F3': 42, 'F#3': 43, 'G3': 44, 'G#3': 45, 'A4': 46,
    'A#4': 47, 'B4': 48, 'C4': 49, 'C#4': 50, 'D4': 51, 'D#4': 52, 'E4': 53, 'F4': 54, 'F#4': 55,
    'G4': 56, 'G#4': 57, 'A5': 58, 'A#5': 59, 'B5': 60, 'C5': 61, 'C#5': 62, 'D5': 63, 'D#5': 64,
    'E5': 65, 'F5': 66, 'F#5': 67, 'G5': 68, 'G#5': 69, 'A6': 70, 'A#6': 71, 'B6': 72, 'C6': 73
}

In [5]:
dataset_1 = [
    (['C1', 'G1', 'A1', 'F1'], (['C1', 'E1', 'G1'], ['G1', 'B2', 'D2'], ['A1', 'C1', 'E1'], ['F1', 'A2', 'C2'])),
    (['C4', 'F4', 'G4', 'A5'], (['C4', 'E4', 'G4'], ['F4', 'C5', 'A5'], ['G4', 'B5', 'D5'], ['A5', 'C5', 'A6'])),
    (['C2', 'D2', 'F2', 'G2'], (['C2', 'G2', 'E3'], ['D2', 'A3', 'F2', 'D3'], ['F2', 'A3', 'C3'], ['G2', 'B3', 'D3']))
]

In [6]:
def encode_notes(notes):
    return [note_mapping[note] for note in notes]

In [7]:
dataset = pd.read_csv("output.csv")

In [9]:
# Process the dataset
X_train = []
y_train = []

#-----------------------------------------------
# for base_notes, chord_progressions in dataset_1:
#     # Encode base notes
#     encoded_base_notes = encode_notes(base_notes)

#     # print(base_notes)

#     # Encode and pad chord progressions
#     encoded_chord_progressions = [encode_notes(chord) for chord in chord_progressions]
#     print(type(encoded_chord_progressions))
#     padded_chord_progressions = pad_sequences(encoded_chord_progressions, maxlen=6, padding='post')

#     # Create input-output pairs
#     X_train.append(encoded_base_notes)
#     y_train.append(padded_chord_progressions)

# X_train = np.array(X_train)
# y_train = np.array(y_train)

# # print("Here's X:")
# # print(X_train)
# # print("Here's y:")
# # print(y_train)
#----------------------------------------------

# # Split the data into training and validation sets
# train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

# # Prepare training data
# X_train = np.array(train_data['base_notes'].tolist())
# y_train = np.array(train_data['chord_progressions'].tolist())

# # Convert string representations to lists
# X_train = [ast.literal_eval(x) for x in X_train]
# y_train = [ast.literal_eval(y) for y in y_train]

# X_train = np.array(X_train)
# y_train = np.array(y_train)

num_lines = 99

for i in range(num_lines):
    base_note = ast.literal_eval(dataset["base_notes"][i])
    chord_progression_raw = ast.literal_eval(dataset["chord_progressions"][i])

    chord_progression = pad_sequences(chord_progression_raw, maxlen=6, padding='post')
    
    X_train.append(base_note)
    y_train.append(chord_progression)

# print(np.array(dataset["base_notes"][99]))

X_train = np.array(X_train)
y_train = np.array(y_train)

# print("Here's X:")
# print(X_train)
# print("Here's y:")
# print(y_train)

In [28]:
# model = Sequential()
# model.add(Dense(units=128, activation = "relu"))
# model.add(Dense(units=4, activation='softmax'))

model = Sequential()
model.add(Dense(units=128, activation="softmax", input_shape=(4,)))
model.add(RepeatVector(4))  # Repeat the vector to match the number of timesteps
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=32, return_sequences=True))
model.add(TimeDistributed(Dense(units=6, activation='softmax')))

In [29]:
model.compile(optimizer=tf.compat.v1.train.AdamOptimizer(), loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
model.fit(X_train, y_train, epochs=100, batch_size=1, validation_split=0.2)

Epoch 1/100
79/79 [==============================] - 1s 9ms/step - loss: 227.9121 - accuracy: 0.7278 - val_loss: 241.4352 - val_accuracy: 0.8375
Epoch 2/100
79/79 [==============================] - 1s 8ms/step - loss: 227.9488 - accuracy: 0.7310 - val_loss: 241.4852 - val_accuracy: 0.6375
Epoch 3/100
79/79 [==============================] - 1s 8ms/step - loss: 228.0468 - accuracy: 0.7152 - val_loss: 241.1183 - val_accuracy: 0.8375
Epoch 4/100
79/79 [==============================] - 1s 8ms/step - loss: 227.9870 - accuracy: 0.7247 - val_loss: 241.5136 - val_accuracy: 0.8375
Epoch 5/100
79/79 [==============================] - 1s 9ms/step - loss: 227.9337 - accuracy: 0.7310 - val_loss: 241.1582 - val_accuracy: 0.8375
Epoch 6/100
79/79 [==============================] - 1s 8ms/step - loss: 227.8886 - accuracy: 0.7342 - val_loss: 242.1936 - val_accuracy: 0.8375
Epoch 7/100
79/79 [==============================] - 1s 9ms/step - loss: 227.9202 - accuracy: 0.7215 - val_loss: 241.5746 - val_ac

In [17]:
print(np.array([1, 5, 6, 4]))

[1 5 6 4]


In [32]:
model.predict(np.array([[11, 51, 24, 46]]))

1/1 [==============================] - 1s 1s/step


array([[[0.28266168, 0.26935023, 0.24484171, 0.18181695, 0.02044935,
         0.00088007],
        [0.30682713, 0.2626866 , 0.23476271, 0.17367081, 0.02059584,
         0.00145697],
        [0.29947054, 0.261554  , 0.23333973, 0.168143  , 0.02955536,
         0.00793731],
        [0.29025185, 0.2583913 , 0.23702428, 0.17670055, 0.02843049,
         0.00920159]]], dtype=float32)